In [1]:
using Documenter;
using JuMP, MathOptInterface;
const MOI = MathOptInterface;
const MOIU = MOI.Utilities;

$$
\begin{array}{rl}
    \max & \mathbf{c}^{\small\mathbf{\text{T}}}\mathbf{x} \\
    \text{s.t.} & \mathbf{w}^{\small\mathbf{\text{T}}}\mathbf{x} +\mathbf{s} = C\\
    ~ & \mathbf{x}_i \in \{0, 1\}
\end{array}
$$

In [2]:
# Model definition
# References:
# [1] https://jump.dev/MathOptInterface.jl/stable/tutorials/example/

model = MOIU.Model{Float64}()

n = 3;
m = 32;
c = [1.0, 2.0, 3.0];
w = [3, 5, 10.];
C = 32.0; #3.2

x = MOI.add_variables(model, n);
s = MOI.add_variables(model, m);

# ---------
# Objective
# ---------
MOI.set(model, MOI.ObjectiveSense(), MOI.MAX_SENSE)

MOI.set(
   model,
   MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(),
   MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(c, x), 0.0),
);

# -----------
# Constraints
# -----------
MOI.add_constraint(
   model,
   MOI.ScalarAffineFunction(vcat(MOI.ScalarAffineTerm.(w, x), MOI.ScalarAffineTerm.(ones(32), s)), 0.0),
   MOI.EqualTo(C),
);

for xᵢ in x
   MOI.add_constraint(model, xᵢ, MOI.ZeroOne())
end

In [74]:
module ToQUBO
    # -*- ToQUBO.jl -*-
    using JuMP, MathOptInterface
    const MOI = MathOptInterface
    const MOIU = MathOptInterface.Utilities
    const MOIB = MathOptInterface.Bridges

    const SVF = MOI.SingleVariable
    const VVF = MOI.VectorOfVariables
    const SAF{T} = MOI.ScalarAffineFunction{T}
    const VAF{T} = MOI.VectorAffineFunction{T}
    const SQF{T} = MOI.ScalarQuadraticFunction{T}
    const VQF{T} = MOI.VectorQuadraticFunction{T}

    const VI = MOI.VariableIndex
    const CI = MOI.ConstraintIndex

    const ZO = MOI.ZeroOne

    # -*- structures.jl -*-
    @doc raw"""
    Storing the problem as pairs {vars => constant} looks like a good a idea.

    $\sum_{\omega \in \Omega} c_\omega \sum_{i \in \omega} x_i$

    Theere are some assumptions about this structure:
        - Variables are all binary $x \in \{0, 1\}$

    [1] Endre Boros, Peter L. Hammer, Pseudo-boolean optimization, 2002
        https://doi.org/10.1016/S0166-218X(01)00341-9
    """
    mutable struct Posiform{T <: Real}
        pairs::Dict{Set{Int}, T}
        degree::Int

        function Posiform{T}() where {T}
            return new{T}(Dict{Set{Int}, T}(), 0)
        end

        function Posiform{T}(x::T) where {T}
            if x == 0
                return new{T}(Dict{Set{Int}, T}(), 0)
            else
                return new{T}(Dict{Set{Int}, T}(Set{Int}() => x), 0)
            end
        end
    
        function Posiform{T}(x::Pair{Set{Int}, T}...) where {T}
            p = Dict{Set{Int}, T}()
        
            for (k, v) in x
                w = get(p, k, 0) + v
                if w == 0
                    delete!(p, k)
                else
                    p[k] = w
                end
            end
        
            d = maximum(length.(keys(p)))
        
            return new{T}(p, d)
        end
    
        function Posiform{T}(x::Dict{Set{Int}, T}) where {T}
            p = Dict{Set{Int}, T}(k => v for (k, v) in x if v != 0)
            d = maximum(length.(keys(p)))
        
            return new{T}(p, d)
        end
    end

    function Base.isempty(p::Posiform{T}) where T
        return isempty(p.pairs)
    end

    function Base.show(io::IO, p::Posiform{T}) where {T}
    
        if isempty(p)
            show(io, "0")
            return
        end
        
        terms = Vector{String}()
    
        for (k, v) in p.pairs
            if isempty(k)
                push!(terms, "$v")
            else
                push!(terms, )
            end
        end
        
        show(io, join(terms, ";"))
        return
    end

    # -*- supported.jl -*-

    # Objective Support
    function supported_objective(model::MOIU.Model{T}) where T
        F = MOI.get(model, MOI.ObjectiveFunctionType())
        if !__supported_objective(F)
            error("Objective functions of type ", F, " are not implemented")
        end
        return
    end

    __supported_objective(::Type) = false

    # Constraint Support
    function supported_constraints(model::MOIU.Model{T}) where T
        for (F, S) in MOI.get(model, MOI.ListOfConstraints())
            if !__supported_constraint(F, S)
                error(
                    "Constraints of function ",
                    F,
                    " in the Set ",
                    S,
                    " are not implemented",
                )
            end
        end
        return
    end

    __supported_constraint(::Type, ::Type) = false

    
    # toqubo
    """
    """
    function toqubo(model::MOIU.Model{T}; quantum::Bool=false) where T
    
        # Support Validation
        #supported_objective(model)
        #supported_constraints(model)

        return __toqubo(model, quantum)
    end;

    """
    """
    function __toqubo(model::MOIU.Model{T}) where T
    
        E = Posiform{T}()
        v = Set{VI}()
    
        # Objective
        F = MOI.get(model, MOI.ObjectiveFunctionType())
    
        # Constraints
    
        for (F, S) in MOI.get(model, MOI.ListOfConstraints())
            # Single Binary Variable
            if F == VI && S == ZO
                for cᵢ in MOI.get(model, MOI.ListOfConstraintIndices{F, S}())
                    vᵢ = MOI.get(model, MOI.ConstraintFunction(), cᵢ)
                    #sᵢ = MOI.get(model, MOI.ConstraintSet(), cᵢ)
                    push!(v, vᵢ)
                end
            end
        end
        
        println("Binary Variables: $v")
    
        println(E)
    
        return model
    end;

    """
    """
    function __toqubo(model::MOIU.Model{T}, quantum::Bool) where T

        qubo_model = __toqubo(model)

        if quantum
            # Objective Sense
            sense = MOI.get(qubo_model, MOI.ObjectiveSense())
            if sense === MOI.MAX_SENSE
                # TODO: Invert Objective Function Signal

                MOI.set(qubo_model, MOI.ObjectiveSense(), MOI.MIN_SENSE)
            elseif sense === MOI.MIN_SENSE
                # Ok!
            else
                error("Invalid Objective Sense '$sense'")
            end
        end

        return qubo_model
    end
end

ToQUBO.toqubo(model)

Binary Variables: Set(MathOptInterface.VariableIndex[MathOptInterface.VariableIndex(2), MathOptInterface.VariableIndex(3), MathOptInterface.VariableIndex(1)])
"0"


MOIU.Model{Float64}

In [75]:
p = ToQUBO.Posiform{Float64}(Set{Int}(1)=>3.0, Set{Int}(2)=>4.5)

""